In [1]:
%matplotlib widget
import numpy as np
import math
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import random

import matplotlib.pyplot as plt

import re

from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

from joblib import Parallel, delayed
import multiprocessing
from datetime import datetime

In [2]:
polskie = open("../polskie-slowa.txt", encoding='utf8')
slowa = [slowo.replace("\n", "") for slowo in polskie.readlines()]
polskie.close()
print(slowa[:2])
print(len(slowa))
slowa = slowa[:1300]

['harcowałabyś', 'groblowych']
13000


In [3]:
chartoidx = {}


cnt = 0

longestword = 0

for slowo in slowa:
    if len(slowo) > longestword:
        longestword = len(slowo)
    
    
    for litera in slowo:
        if litera not in list(chartoidx.keys()):
            chartoidx[litera] = cnt
            cnt = cnt + 1
            
            
chartoidx["<END>"] = cnt
# chartoidx["<START>"] = cnt+1

In [6]:
chartoidx

{'h': 0,
 'a': 1,
 'r': 2,
 'c': 3,
 'o': 4,
 'w': 5,
 'ł': 6,
 'b': 7,
 'y': 8,
 'ś': 9,
 'g': 10,
 'l': 11,
 'n': 12,
 'ż': 13,
 'i': 14,
 'z': 15,
 's': 16,
 'm': 17,
 'd': 18,
 'ń': 19,
 'e': 20,
 'j': 21,
 't': 22,
 'k': 23,
 'u': 24,
 'ą': 25,
 'p': 26,
 'ó': 27,
 'ę': 28,
 'f': 29,
 'v': 30,
 'ć': 31,
 'ź': 32,
 '<END>': 33}

In [4]:
windowsize = longestword

In [5]:
np.zeros((1, len(list(chartoidx.keys())) ))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])

In [6]:
def word_to_window(word, windowsize, chartoidx):
    oknox = []
    oknoy = []
    
    zeros = [0 for __  in range(len(list(chartoidx.keys() )))]
    
    for cnt1, literka in enumerate(word):
        slowoins = []
        for cnt2 in range(windowsize):
            if cnt2 <= cnt1:
                translated = zeros.copy()
                translated[chartoidx[word[cnt2]]] = 1
                
                slowoins.append(translated)
            else:
                slowoins.append(zeros)
                
        if cnt1+1 < len(word): 
            oknoy.append(chartoidx[word[cnt1+1]])
        else:
            oknoy.append(chartoidx["<END>"])
                
                
        oknox.append(slowoins)
        
        
    
    return oknox, oknoy

In [10]:
word_to_window("abmicro", windowsize, chartoidx)

([[[0,
    1,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0],
   [0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0],
   [0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0],
   [0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0],
   [0,
    0,
    0,
    0,
    0,
    0,
  

In [34]:
def word_to_window(word, windowsize, chartoidx):
    oknox = []
    oknoy = []
    
    zeros = [0 for __  in range(len(list(chartoidx.keys() )))]
    
    for cnt in range(windowsize):
        if cnt == 0:
            translated = zeros.copy()
            translated[chartoidx["<START>"]] = 1
            
            oknox.append(translated)            
            oknoy.append(chartoidx[word[cnt]])
        elif cnt < (len(word) - 1):
            translated = zeros.copy()
            translated[chartoidx[word[cnt]]] = 1
            
            oknox.append(translated)
            oknoy.append(chartoidx[word[cnt+1]])
        elif cnt == (len(word) -1):
            translated = zeros.copy()
            translated[chartoidx[word[cnt]]] = 1
            
            oknox.append(translated)
            oknoy.append(chartoidx["<END>"])
        else:
            oknox.append(zeros)
            oknoy.append(zeros)    
    
    return [oknox], [oknoy]

In [35]:
word_to_window("abmicro", 7, chartoidx)

([[[0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    1],
   [0,
    0,
    0,
    0,
    0,
    0,
    0,
    1,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0],
   [0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    1,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0],
   [0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    1,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0],
   [0,
    0,
  

In [7]:
DatasetX = []
DatasetY = []

for slowo in slowa:
    x, y = word_to_window(slowo, windowsize, chartoidx)
    
    DatasetX = DatasetX + x
    DatasetY = DatasetY + y
    
DatasetX = np.array(DatasetX)
DatasetY = np.array(DatasetY)

In [8]:
DatasetX.shape

(14581, 15, 34)

In [9]:
DatasetY.shape

(14581,)

In [10]:
DatasetY

array([ 1,  2,  3, ..., 17, 24, 33])

In [11]:
TensorX = torch.Tensor(DatasetX).cuda()

In [12]:
#TensorX

In [13]:
def EmbeddingDrop(dataset, p=0.1):
    w1, w2, w3 = dataset.shape
    bern = torch.bernoulli((1-p) * torch.ones((w1, w2, 1)).cuda())
    # print(bern)
    return dataset * bern

In [14]:
#EmbeddingDrop(TensorX, p=0.4)

In [15]:
torch.index_select(TensorX, 1, torch.Tensor([0]).long().cuda()).shape

torch.Size([14581, 1, 34])

In [16]:
word_dimensions = longestword
_, _, words_dimension = TensorX.shape

print("ilosc modulow:", word_dimensions)

ilosc modulow: 15


In [17]:
word_dimensions

15

In [18]:
class CharacterLSTM(nn.Module):
    def __init__(self, word_dimensions, words_dimension):
        super(CharacterLSTM, self).__init__()
        
        self.word_dimensions = word_dimensions
        self.words_dimension = words_dimension
        
        self.LSTM_11 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_12 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_13 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_14 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_15 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_16 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_17 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_18 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_19 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_110 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_111 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_112 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_113 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_114 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_115 = nn.LSTM(words_dimension, words_dimension)
        
        
        self.LSTM_21 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_22 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_23 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_24 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_25 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_26 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_27 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_28 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_29 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_210 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_211 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_212 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_213 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_214 = nn.LSTM(words_dimension, words_dimension)
        self.LSTM_215 = nn.LSTM(words_dimension, words_dimension)
        
        
        self.norm = nn.LayerNorm(words_dimension)
        self.drop = nn.Dropout(p=0.1)
        
        
        self.linear_ins = nn.Linear(words_dimension * word_dimensions, words_dimension * word_dimensions)
        self.linear_outs = nn.Linear(words_dimension * word_dimensions, words_dimension)
        
    def forward(self, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, hidden, drop = True):
       
        if drop == True:
            hidd = (self.EmbeddingDrop(hidden[0], p=0.1), self.EmbeddingDrop(hidden[1], p=0.1))
        else:
            hidd = hidden
        
        # pierwsza warstwa LSTM
        y11_, h11_ = self.LSTM_11(x1, hidd)
        y12_, h12_ = self.LSTM_12(x2, h11_)
        y13_, h13_ = self.LSTM_13(x3, h12_)
        y14_, h14_ = self.LSTM_14(x4, h13_)
        y15_, h15_ = self.LSTM_15(x5, h14_)
        y16_, h16_ = self.LSTM_16(x6, h15_)
        y17_, h17_ = self.LSTM_17(x7, h16_)
        y18_, h18_ = self.LSTM_18(x8, h17_)
        y19_, h19_ = self.LSTM_19(x9, h18_)
        y110_, h110_ = self.LSTM_110(x10, h19_)
        y111_, h111_ = self.LSTM_111(x11, h110_)
        y112_, h112_ = self.LSTM_112(x12, h111_)
        y113_, h113_ = self.LSTM_113(x13, h112_)
        y114_, h114_ = self.LSTM_114(x14, h113_)
        y115_, h115_ = self.LSTM_115(x15, h114_)
        
        
        # residuals
        if drop == True:
            y11 = self.norm(self.EmbeddingDrop(y11_ + x1))
            y12 = self.norm(self.EmbeddingDrop(y12_ + x2))
            y13 = self.norm(self.EmbeddingDrop(y13_ + x3))
            y14 = self.norm(self.EmbeddingDrop(y14_ + x4))
            y15 = self.norm(self.EmbeddingDrop(y15_ + x5))
            y16 = self.norm(self.EmbeddingDrop(y16_ + x6))
            y17 = self.norm(self.EmbeddingDrop(y17_ + x7))
            y18 = self.norm(self.EmbeddingDrop(y18_ + x8))
            y19 = self.norm(self.EmbeddingDrop(y19_ + x9))
            y110 = self.norm(self.EmbeddingDrop(y110_ + x10))
            y111 = self.norm(self.EmbeddingDrop(y111_ + x11))
            y112 = self.norm(self.EmbeddingDrop(y112_ + x12))
            y113 = self.norm(self.EmbeddingDrop(y113_ + x13))
            y114 = self.norm(self.EmbeddingDrop(y114_ + x14))
            y115 = self.norm(self.EmbeddingDrop(y115_ + x15))
        else:
            y11 = self.norm(y11_ + x1)
            y12 = self.norm(y12_ + x2)
            y13 = self.norm(y13_ + x3)
            y14 = self.norm(y14_ + x4)
            y15 = self.norm(y15_ + x5)
            y16 = self.norm(y16_ + x6)
            y17 = self.norm(y17_ + x7)
            y18 = self.norm(y18_ + x8)
            y19 = self.norm(y19_ + x9)
            y110 = self.norm(y110_ + x10)
            y111 = self.norm(y111_ + x11)
            y112 = self.norm(y112_ + x12)
            y113 = self.norm(y113_ + x13)
            y114 = self.norm(y114_ + x14)
            y115 = self.norm(y115_ + x15)
        
        
        # druga warstwa LSTM
        y21_, h21_ = self.LSTM_21(y11, h115_)
        y22_, h22_ = self.LSTM_22(y12, h21_)
        y23_, h23_ = self.LSTM_23(y13, h22_)
        y24_, h24_ = self.LSTM_24(y14, h23_)
        y25_, h25_ = self.LSTM_25(y15, h24_)
        y26_, h26_ = self.LSTM_26(y16, h25_)
        y27_, h27_ = self.LSTM_27(y17, h26_)
        y28_, h28_ = self.LSTM_28(y18, h27_)
        y29_, h29_ = self.LSTM_28(y19, h28_)
        y210_, h210_ = self.LSTM_210(y110, h28_)
        y211_, h211_ = self.LSTM_211(y111, h210_)
        y212_, h212_ = self.LSTM_212(y112, h211_)
        y213_, h213_ = self.LSTM_213(y113, h212_)
        y214_, h214_ = self.LSTM_214(y114, h213_)
        y215_, h215_ = self.LSTM_215(y115, h214_)
        
        
        # residuals
#         y21 = self.norm(y21_ + y11 + x1)
#         y22 = self.norm(y22_ + y12 + x2)
#         y23 = self.norm(y23_ + y13 + x3)
#         y24 = self.norm(y24_ + y14 + x4)
#         y25 = self.norm(y25_ + y15 + x5)
#         y26 = self.norm(y26_ + y16 + x6)
#         y27 = self.norm(y27_ + y17 + x7)
#         y28 = self.norm(y28_ + y18 + x8)
#         y29 = self.norm(y29_ + y19 + x9)
#         y210 = self.norm(y210_ + y110 + x10)
#         y211 = self.norm(y211_ + y111 + x11)
#         y212 = self.norm(y212_ + y112 + x12)
#         y213 = self.norm(y213_ + y113 + x13)
#         y214 = self.norm(y214_ + y114 + x14)
#         y215 = self.norm(y215_ + y115 + x15)
        if drop == True:
            y21 = self.norm(self.EmbeddingDrop(y21_ + y11))
            y22 = self.norm(self.EmbeddingDrop(y22_ + y12))
            y23 = self.norm(self.EmbeddingDrop(y23_ + y13))
            y24 = self.norm(self.EmbeddingDrop(y24_ + y14))
            y25 = self.norm(self.EmbeddingDrop(y25_ + y15))
            y26 = self.norm(self.EmbeddingDrop(y26_ + y16))
            y27 = self.norm(self.EmbeddingDrop(y27_ + y17))
            y28 = self.norm(self.EmbeddingDrop(y28_ + y18))
            y29 = self.norm(self.EmbeddingDrop(y29_ + y19))
            y210 = self.norm(self.EmbeddingDrop(y210_ + y110))
            y211 = self.norm(self.EmbeddingDrop(y211_ + y111))
            y212 = self.norm(self.EmbeddingDrop(y212_ + y112))
            y213 = self.norm(self.EmbeddingDrop(y213_ + y113))
            y214 = self.norm(self.EmbeddingDrop(y214_ + y114))
            y215 = self.norm(self.EmbeddingDrop(y215_ + y115))
        else:
            y21 = self.norm(y21_ + y11)
            y22 = self.norm(y22_ + y12)
            y23 = self.norm(y23_ + y13)
            y24 = self.norm(y24_ + y14)
            y25 = self.norm(y25_ + y15)
            y26 = self.norm(y26_ + y16)
            y27 = self.norm(y27_ + y17)
            y28 = self.norm(y28_ + y18)
            y29 = self.norm(y29_ + y19)
            y210 = self.norm(y210_ + y110)
            y211 = self.norm(y211_ + y111)
            y212 = self.norm(y212_ + y112)
            y213 = self.norm(y213_ + y113)
            y214 = self.norm(y214_ + y114)
            y215 = self.norm(y215_ + y115)

        
        
        # linear
        
        y_lin_cat = torch.cat((y21, y22, y23, y24, y25, y26, y27, y28, y29, y210, y211, y212, y213, y214, y215), 1)
        if drop == True:
            y30_ = self.drop(y_lin_cat)
        else:
            y30_ = y_lin_cat
        
        y31 = F.relu(self.linear_ins(y_lin_cat.reshape(len(x1), -1)))
        
        if drop == True:
            y31_ = self.drop(y31)
        else:
            y31_ = y31
        
        y32 = F.relu(self.linear_outs(y31_))
        
        mm = nn.LogSoftmax(dim=1)
        
        y32_ = mm(y32)
                
        
        # zwrot
        return y32_
    
    def EmbeddingDrop(self, dataset, p=0.1):
        w1, w2, w3 = dataset.shape
        bern = torch.bernoulli((1-p) * torch.ones((w1, w2, 1)).cuda())
        # print(bern)
        return dataset * bern

In [19]:
chlstm = CharacterLSTM(word_dimensions, words_dimension).cuda()

In [78]:
chlstm = torch.load("model_chlstm.pt").cuda()

In [20]:
# x1 = torch.index_select(TensorX, 1, torch.Tensor([0]).long().cuda())
# x2 = torch.index_select(TensorX, 1, torch.Tensor([1]).long().cuda())
# x3 = torch.index_select(TensorX, 1, torch.Tensor([2]).long().cuda())
# x4 = torch.index_select(TensorX, 1, torch.Tensor([3]).long().cuda())
# x5 = torch.index_select(TensorX, 1, torch.Tensor([4]).long().cuda())
# x6 = torch.index_select(TensorX, 1, torch.Tensor([5]).long().cuda())
# x7 = torch.index_select(TensorX, 1, torch.Tensor([6]).long().cuda())
# x8 = torch.index_select(TensorX, 1, torch.Tensor([7]).long().cuda())
# x9 = torch.index_select(TensorX, 1, torch.Tensor([8]).long().cuda())
# x10 = torch.index_select(TensorX, 1, torch.Tensor([9]).long().cuda())
# x11 = torch.index_select(TensorX, 1, torch.Tensor([10]).long().cuda())
# x12 = torch.index_select(TensorX, 1, torch.Tensor([11]).long().cuda())
# x13 = torch.index_select(TensorX, 1, torch.Tensor([12]).long().cuda())
# x14 = torch.index_select(TensorX, 1, torch.Tensor([13]).long().cuda())
# x15 = torch.index_select(TensorX, 1, torch.Tensor([14]).long().cuda())

In [21]:
# print(x1.shape, x2.shape, x3.shape, x4.shape, x5.shape, x6.shape, x7.shape, x8.shape, x9.shape)

In [24]:
# hiddens = (torch.rand(1, 1, words_dimension).cuda(), torch.rand(1, 1, words_dimension).cuda())

In [25]:
hiddens = torch.load("model_chlstm_hiddens.pt")

In [26]:
# x1 = torch.index_select(dropped, 1, torch.Tensor([0]).long().cuda())
# x2 = torch.index_select(dropped, 1, torch.Tensor([1]).long().cuda())
# x3 = torch.index_select(dropped, 1, torch.Tensor([2]).long().cuda())
# x4 = torch.index_select(dropped, 1, torch.Tensor([3]).long().cuda())
# x5 = torch.index_select(dropped, 1, torch.Tensor([4]).long().cuda())
# x6 = torch.index_select(dropped, 1, torch.Tensor([5]).long().cuda())
# x7 = torch.index_select(dropped, 1, torch.Tensor([6]).long().cuda())
# x8 = torch.index_select(dropped, 1, torch.Tensor([7]).long().cuda())
# x9 = torch.index_select(dropped, 1, torch.Tensor([8]).long().cuda())
# x10 = torch.index_select(dropped, 1, torch.Tensor([9]).long().cuda())
# x11 = torch.index_select(dropped, 1, torch.Tensor([10]).long().cuda())
# x12 = torch.index_select(dropped, 1, torch.Tensor([11]).long().cuda())
# x13 = torch.index_select(dropped, 1, torch.Tensor([12]).long().cuda())
# x14 = torch.index_select(dropped, 1, torch.Tensor([13]).long().cuda())
# x15 = torch.index_select(dropped, 1, torch.Tensor([14]).long().cuda())

# wynik = chlstm(x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, hiddens)

In [28]:
# wynik.shape

In [29]:
criterion = nn.NLLLoss()

In [30]:
# criterion = nn.CrossEntropyLoss()

In [31]:
TorchY = torch.Tensor(DatasetY).long().cuda()

In [32]:
# criterion(wynik, TorchY)

In [33]:
optimizer = optim.Adam(chlstm.parameters(), lr=4e-3)

In [34]:
for epoch in range(30000):
    chlstm.train()
    optimizer.zero_grad()
    
    
    dropped = EmbeddingDrop(TensorX, p=0.1)
    
    x1 = torch.index_select(dropped, 1, torch.Tensor([0]).long().cuda())
    x2 = torch.index_select(dropped, 1, torch.Tensor([1]).long().cuda())
    x3 = torch.index_select(dropped, 1, torch.Tensor([2]).long().cuda())
    x4 = torch.index_select(dropped, 1, torch.Tensor([3]).long().cuda())
    x5 = torch.index_select(dropped, 1, torch.Tensor([4]).long().cuda())
    x6 = torch.index_select(dropped, 1, torch.Tensor([5]).long().cuda())
    x7 = torch.index_select(dropped, 1, torch.Tensor([6]).long().cuda())
    x8 = torch.index_select(dropped, 1, torch.Tensor([7]).long().cuda())
    x9 = torch.index_select(dropped, 1, torch.Tensor([8]).long().cuda())
    x10 = torch.index_select(dropped, 1, torch.Tensor([9]).long().cuda())
    x11 = torch.index_select(dropped, 1, torch.Tensor([10]).long().cuda())
    x12 = torch.index_select(dropped, 1, torch.Tensor([11]).long().cuda())
    x13 = torch.index_select(dropped, 1, torch.Tensor([12]).long().cuda())
    x14 = torch.index_select(dropped, 1, torch.Tensor([13]).long().cuda())
    x15 = torch.index_select(dropped, 1, torch.Tensor([14]).long().cuda())
    
    wynik = chlstm(x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, hiddens)
    loss = criterion(wynik, TorchY)
    
    loss.backward()
    optimizer.step()
    
    if epoch%10==0:
        print(loss.item())
        
#     if epoch % 50 == 0:
#         torch.save(chlstm, "model_chlstm_"+str(epoch)+".pt")

3.5279347896575928
3.2268826961517334
3.146937370300293
3.0842320919036865
3.0088021755218506
2.8722641468048096
2.776287794113159
2.6858890056610107
2.6046524047851562
2.547915458679199
2.4589924812316895
2.390916109085083
2.322190999984741
2.2588069438934326
2.2072322368621826
2.1466734409332275
2.0852770805358887
2.047699213027954
2.0088882446289062
1.9557335376739502
1.927262544631958
1.8966721296310425
1.864833116531372
1.8437838554382324
1.815801978111267
1.7807729244232178
1.7748167514801025
1.74560546875
1.738568663597107
1.717638373374939
1.689974308013916
1.6819334030151367
1.6657655239105225
1.6514006853103638
1.632786750793457
1.6267813444137573
1.6217738389968872
1.592552661895752
1.5925614833831787
1.5697975158691406
1.560988426208496
1.5483791828155518
1.5424447059631348
1.5274560451507568
1.5322034358978271
1.5063211917877197
1.511894702911377
1.5014938116073608
1.4920941591262817
1.4719953536987305
1.4701958894729614
1.4681801795959473
1.4579414129257202
1.440997958183

KeyboardInterrupt: 

In [35]:
torch.save(chlstm, "model_chlstm_LOG.pt")

C:\Anaconda3\lib\site-packages\torch\serialization.py:251: UserWarning: Couldn't retrieve source code for container of type CharacterLSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [36]:
torch.save(hiddens, "model_chlstm_hiddens_LOG.pt")

In [102]:
def word_to_predict(word, windowsize, chartoidx):
    oknox = []
    
    zeros = [0 for __  in range(len(list(chartoidx.keys() )))]
    
    for cnt2 in range(windowsize):
        if cnt2 < len(word):
            translated = zeros.copy()
            translated[chartoidx[word[cnt2]]] = 1

            oknox.append(translated)
        else:
            oknox.append(zeros)
        
        
    
    return np.array(oknox).reshape(windowsize, -1)

In [107]:
slowo = "mak"
znak = ""




chlstm.eval()

while znak != "<END>":
    with torch.no_grad():
        slowo_coded = word_to_predict(slowo, windowsize, chartoidx)
        
        word = torch.Tensor(slowo_coded).cuda().reshape(1, windowsize, -1)

        x1 = torch.index_select(word, 1, torch.Tensor([0]).long().cuda())
        x2 = torch.index_select(word, 1, torch.Tensor([1]).long().cuda())
        x3 = torch.index_select(word, 1, torch.Tensor([2]).long().cuda())
        x4 = torch.index_select(word, 1, torch.Tensor([3]).long().cuda())
        x5 = torch.index_select(word, 1, torch.Tensor([4]).long().cuda())
        x6 = torch.index_select(word, 1, torch.Tensor([5]).long().cuda())
        x7 = torch.index_select(word, 1, torch.Tensor([6]).long().cuda())
        x8 = torch.index_select(word, 1, torch.Tensor([7]).long().cuda())
        x9 = torch.index_select(word, 1, torch.Tensor([8]).long().cuda())
        x10 = torch.index_select(word, 1, torch.Tensor([9]).long().cuda())
        x11 = torch.index_select(word, 1, torch.Tensor([10]).long().cuda())
        x12 = torch.index_select(word, 1, torch.Tensor([11]).long().cuda())
        x13 = torch.index_select(word, 1, torch.Tensor([12]).long().cuda())
        x14 = torch.index_select(word, 1, torch.Tensor([13]).long().cuda())
        x15 = torch.index_select(word, 1, torch.Tensor([14]).long().cuda())

        wynik = chlstm(x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, hiddens, drop=False)

        softmaxed = F.softmax(wynik, 1)

        znak = list(chartoidx.keys())[torch.argmax(softmaxed).item()]

        slowo = slowo + znak
        
slowo

'makrhhrrhaa<END>'